In [1]:
import numpy as np
import tensorflow as tf
from random import shuffle
#https://gist.github.com/danijar/d11c77c5565482e965d1919291044470
#https://github.com/crestonbunch/neural-namer/blob/master/modeler/network.py
#https://danijar.com/variable-sequence-lengths-in-tensorflow/

In [2]:
data = open('dinos.txt', 'r').read()
data = data.lower()
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('There are %d total characters and %d unique characters in your data.' % (data_size, vocab_size))

There are 19909 total characters and 27 unique characters in your data.


In [3]:
char_to_ix = { ch:i for i,ch in enumerate(sorted(chars)) }
ix_to_char = { i:ch for i,ch in enumerate(sorted(chars)) }
print(ix_to_char)

{0: '\n', 1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z'}


In [4]:
def word_to_one_vector(word, char_to_ix, paddTo = 0):
    one_hot_vectors = np.zeros((len(word), len(char_to_ix)))
    one_hot_vectors[ np.arange(len(word)) , [char_to_ix[ch] for ch in word] ]= 1
    if paddTo > 0:
        one_hot_vectors = np.vstack((one_hot_vectors, np.zeros((paddTo - len(word), len(char_to_ix)))))
    return one_hot_vectors

In [5]:
with open("dinos.txt") as f:
    examples = f.readlines()
examples = [x.lower().strip() for x in examples]

max_length = 0
for x in examples:
    if len(x) > max_length:
        max_length = len(x)
print(word_to_one_vector(examples[0], char_to_ix))

[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.
  0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
  0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
  0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

In [6]:
n_neurons = 200

In [7]:
X = tf.placeholder(tf.float32, [None, None, vocab_size])
y = tf.placeholder(tf.int32, [None, None]) #Shape => Batch_Size x Steps
seq_len = tf.placeholder(tf.int32)

In [8]:
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units =  n_neurons)
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype = tf.float32, sequence_length = seq_len)

In [9]:
stacked_outputs = tf.reshape(outputs, [-1, n_neurons])
stacked_outputs_dense = tf.layers.dense(stacked_outputs, vocab_size)
outputs_2 = tf.reshape(stacked_outputs_dense, [-1, seq_len, vocab_size])

In [10]:
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits = outputs_2)
loss = tf.reduce_mean(xentropy)
optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.01)
training_op = optimizer.minimize(loss)

In [11]:
init = tf.global_variables_initializer()

In [13]:
with tf.Session() as sess:
    sess.run(init)
    iteration = 0
    for epoch in range(50000):
        for example  in examples:
            x_ = np.vstack((np.zeros(len(char_to_ix)),word_to_one_vector(example, char_to_ix, max_length)))
            x_ = x_.reshape(1, x_.shape[0], x_.shape[1])
            y_ = np.append(np.asarray([(char_to_ix[x]) for x in example]),0)
            y_ = y_.reshape(1, y_.shape[0])
            sess.run(training_op, feed_dict = {X: x_, y: y_, seq_len: x_.shape[1]})
            #out = sess.run(outputs_2, feed_dict = {X: x_, y: y_, seq_len: x_.shape[1]})
            #print(out.shape, y_)
            if iteration % 1000 == 0:
                print(loss.eval(feed_dict = {X: x_, y: y_, seq_len: x_.shape[1]}))
                word = []
                x = np.zeros((vocab_size)).reshape(-1, 1, vocab_size)
                for iter_number in range(50):
                    out = sess.run(outputs_2, feed_dict = {X: x, seq_len: 1})
                    indexs = tf.nn.softmax(out).eval().reshape(-1, vocab_size)
                    choice = np.random.choice(range(vocab_size), p = indexs[0])
                    if choice == 0:
                        break
                    word.append(ix_to_char[choice])
                    x = np.zeros((vocab_size))
                    x[choice] = 1
                    x = x.reshape(-1, 1, vocab_size)
                word = ''.join(word) 
                print(iteration, word)
            iteration += 1
        shuffle(examples)


InvalidArgumentError: assertion failed: [] [Condition x == y did not hold element-wise:] [x (SparseSoftmaxCrossEntropyWithLogits/Shape_1:0) = ] [1 10] [y (SparseSoftmaxCrossEntropyWithLogits/strided_slice:0) = ] [1 27]
	 [[Node: SparseSoftmaxCrossEntropyWithLogits/assert_equal/Assert/Assert = Assert[T=[DT_STRING, DT_STRING, DT_STRING, DT_INT32, DT_STRING, DT_INT32], summarize=3, _device="/job:localhost/replica:0/task:0/device:CPU:0"](SparseSoftmaxCrossEntropyWithLogits/assert_equal/All, SparseSoftmaxCrossEntropyWithLogits/assert_equal/Assert/Assert/data_0, SparseSoftmaxCrossEntropyWithLogits/assert_equal/Assert/Assert/data_1, SparseSoftmaxCrossEntropyWithLogits/assert_equal/Assert/Assert/data_2, SparseSoftmaxCrossEntropyWithLogits/Shape, SparseSoftmaxCrossEntropyWithLogits/assert_equal/Assert/Assert/data_4, SparseSoftmaxCrossEntropyWithLogits/strided_slice)]]

Caused by op 'SparseSoftmaxCrossEntropyWithLogits/assert_equal/Assert/Assert', defined at:
  File "/usr/lib/python3.6/runpy.py"